# Génération des codes et formattage des données avant insertion en base

## Traitements


1. Chargement de la bdd existante et de la nouvelle généré à l'étape 1 (au même format)
2. Formattage de certaines colonnes & ajout d'autres pour la base de données ('zrr', 'qpv', 'a_valider', 'refuser')
3. Génération des codes manquants pour les nouvelles lignes de bénéficiaires
4. Output to csv

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import json
import numpy as np

load_dotenv()

new_db_export_filepath = os.environ['DB_EXPORT']
concatenated_db_filepath = os.environ['DB_CONCATENATED']
final_merged_with_ids = os.environ['DB_MERGED_WITH_ID']
final_merged_without_ids = os.environ['DB_MERGED_WITHOUT_ID']

In [ ]:
# Loading new data
df_new_db = pd.read_csv(new_db_export_filepath, index_col=0, sep=',')

In [ ]:
# Convert to date for comparison, timestamp is added back later down the line
df_new_db['date_naissance'] = pd.to_datetime(df_new_db['date_naissance']).dt.date

In [ ]:
# json unwrap allocatire
df_json_allocataire_new = pd.json_normalize(df_new_db['allocataire'].apply(json.loads))
df_json_allocataire_new = df_json_allocataire_new.add_prefix('allocataire-')

df_new_db.index = pd.RangeIndex(start=0, stop=len(df_new_db), step=1)
df_new_db_unwrapped_alloc = pd.merge(df_new_db, df_json_allocataire_new, left_index=True, right_index=True)

df_new_db_unwrapped_alloc = df_new_db_unwrapped_alloc.drop(columns=['allocataire'])

In [ ]:
# Data casting and formating
df_new_db_unwrapped_alloc['nom'] = df_new_db_unwrapped_alloc['nom'].str.upper()
df_new_db_unwrapped_alloc['prenom'] = df_new_db_unwrapped_alloc['prenom'].str.upper()

mask_matricule_not_null = df_new_db_unwrapped_alloc['allocataire-matricule'].notna()
df_new_db_unwrapped_alloc['allocataire-matricule'] = df_new_db_unwrapped_alloc.loc[mask_matricule_not_null, 'allocataire-matricule'].astype(str)

# replace blank string with nan
df_new_db_unwrapped_alloc = df_new_db_unwrapped_alloc.replace(r'', np.NaN)

In [ ]:
import pytz
import datetime

# add created_at for new rows
# using time zone for created_at to be iso existings
tz = pytz.timezone('Europe/Paris')
now = datetime.datetime.now()
now_tz = tz.localize(now)

df_new_db_unwrapped_alloc['created_at'] = now_tz
df_new_db_unwrapped_alloc[['zrr', 'qpv', 'a_valider', 'refuser']] = False

In [ ]:
# map to json values for target DB model 
def to_json_allocataire_without_null(row):
    allocataire_mapping = {
        'qualite': row['allocataire-qualite'],
        'matricule': row['allocataire-matricule'],
        'nom': row['allocataire-nom'],
        'prenom': row['allocataire-prenom'],
        'date_naissance': row['allocataire-date_naissance'],
        'courriel': row['allocataire-courriel'],
        'code_insee_commune_naissance': row['allocataire-code_insee_commune_naissance'],
        'commune_naissance': row['allocataire-commune_naissance'],
        'code_iso_pays_naissance': row['allocataire-code_iso_pays_naissance'],
        'pays_naissance': row['allocataire-pays_naissance']
    }
    filtered_nan_allocataire = {k: v for k, v in allocataire_mapping.items() if pd.notnull(v)}
    return json.dumps(filtered_nan_allocataire, ensure_ascii=False)

df_new_db_unwrapped_alloc['allocataire'] = df_new_db_unwrapped_alloc.apply(to_json_allocataire_without_null, axis=1)

In [ ]:
df_final_with_code = df_new_db_unwrapped_alloc.drop(columns=[
  'allocataire-qualite',
  'allocataire-matricule',
  'allocataire-nom',
  'allocataire-prenom',
  'allocataire-date_naissance',
  'allocataire-courriel',
  'allocataire-code_insee_commune_naissance',
  'allocataire-commune_naissance',
  'allocataire-code_iso_pays_naissance',
  'allocataire-pays_naissance',
])

In [ ]:
# Add back timestamp and the 4 hours to be iso with the existing database
df_final_with_code['date_naissance'] = pd.to_datetime(df_final_with_code['date_naissance']).dt.floor('D') + pd.DateOffset(hours=4)

In [ ]:
df_final_with_code.to_csv(final_merged_without_ids, index=False)